In [307]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from difflib import get_close_matches
from nltk.stem import SnowballStemmer

# Télécharger les ressources nécessaires de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Houssam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Houssam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Houssam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Exercice 1

In [308]:

abreviations = {
    "BID": "bis in die",
    "IV": "intraveineuse",
    'Ttt' : "traitement",
    'Pat.' : 'patient',
    "corticoïdes" : "glucocorticoides",
    "++fébrile" : "hyperthermique"
}

def clean(text:str):
    
    for abreviation, signification in abreviations.items():
        text = text.replace(abreviation, signification)
    
    text = re.sub(r'\s-\s', ' ', text)

    text = re.sub(r'\d{2}/\d{2}/\d{2,4}', '', text)

    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

    text = re.sub(r'[^\w\s.-]', '', text)

    text = re.sub(r'\d+(\.\d+)?[a-zA-Z/]+', '', text)

    text = re.sub(r'\n', ' ', text)

    text = re.sub(r'\s{2,}', ' ', text)

    return text.strip()


In [309]:
texte_medical = """
Pat. 45ans admis le 12/03/25 pour COVID-19 sévère.
CRP: 125mg/L (N<5) - Fièvre à 39.2°C.
Ttt par corticoïdes IV + azithromycine 500mg/J.
#Attention: allergie à la pénicilline!
"""

In [310]:
cleaned_text = clean(texte_medical)
print(cleaned_text)

patient admis le pour COVID-19 sévère CRP N5 Fièvre à traitement par glucocorticoides intraveineuse azithromycine Attention allergie à la pénicilline


## Exercice 2

In [311]:
def tokenisation(text:str):
    for abreviation, signification in abreviations.items():
        text = text.replace(abreviation, signification)

    text = re.sub(r'\s-\s', ' ', text)

    text = re.sub(r'\d{2}/\d{2}/\d{2,4}', '', text)

    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

    text = re.sub(r'[^\w\s./<-]', '', text)

    text = re.sub(r'\n', ' ', text)

    text = re.sub(r'\s{2,}', ' ', text)

    text = re.sub(r'(\d+)([a-zA-Z]+)', r'\1 \2', text)  # sépare nombre et unité
    text = re.sub(r'([a-zA-Z]+)(/)([a-zA-Z]+)', r'\1 \2 \3', text)  # sépare unités autour du /

    mots = word_tokenize(text)

    return mots

    

In [312]:
tokenized = tokenisation(texte_medical)
print(tokenized)

['patient', '45', 'ans', 'admis', 'le', 'pour', 'COVID-19', 'sévère', 'CRP', '125', 'mg', '/', 'L', 'N', '<', '5', 'Fièvre', 'à', '39.2', 'C', 'traitement', 'par', 'glucocorticoides', 'intraveineuse', 'azithromycine', '500', 'mg', '/', 'J', 'Attention', 'allergie', 'à', 'la', 'pénicilline']


In [313]:
def supprimer_stopwords(mots):
    stop_words = set(stopwords.words('french'))

    mots_filtres = [mot.lower() for mot in mots if mot not in stop_words]
    return mots_filtres

In [314]:
no_stop_words = supprimer_stopwords(tokenized)
print(no_stop_words)

['patient', '45', 'ans', 'admis', 'covid-19', 'sévère', 'crp', '125', 'mg', '/', 'l', 'n', '<', '5', 'fièvre', '39.2', 'c', 'traitement', 'glucocorticoides', 'intraveineuse', 'azithromycine', '500', 'mg', '/', 'j', 'attention', 'allergie', 'pénicilline']


## Exercice 3

In [315]:
# Exemple mini dictionnaire médical
dictionnaire_medical = [
    "hypoglycémie", "diabète", "hypertension", "tachycardie", "fièvre", "asthme"
]

def corriger(mot: str):
    if mot.isdigit():
        return mot
    
    corrections = {
        '0': 'o', '1': 'l', '5': 's', '6': 'b', '€': 'e'
    }
    for faute, correct in corrections.items():
        mot = mot.replace(faute, correct)

    # Correction par similarité sur le dictionnaire
    suggestions = get_close_matches(mot, dictionnaire_medical, n=1, cutoff=0.8)
    if suggestions:
        return suggestions[0]
    else:
        return mot


In [316]:
def normalisation(words):
    corrected = [corriger(word) for word in words]

    # Initialiser le stemmer pour le français
    stemmer = SnowballStemmer("french")
    # Appliquer le stemming à chaque mot
    mots_normalises = [stemmer.stem(mot) for mot in corrected]
    return mots_normalises

In [317]:
print(normalisation(no_stop_words))

['patient', '45', 'an', 'admis', 'covid-l9', 'séver', 'crp', '125', 'mg', '/', 'l', 'n', '<', '5', 'fievr', '39.2', 'c', 'trait', 'glucocorticoid', 'intravein', 'azithromycin', '500', 'mg', '/', 'j', 'attent', 'allerg', 'pénicillin']


## Exemples

In [318]:
data = {
    "texte1" : """Pat. 60ans hospitalisé le 15/04/2023 pour pneumopathie.
        CRP à 240mg/L. Ttt initial par amoxicilline 1g/J.
        Fièvre persistante à 38.5°C. Scanner réalisé.
        """,
    "texte2" : """Pat. 33ans. Chute le 01/01/25. Scanner cérébral normal.
        Température à 37.8ºC. Ttt: paracétamol 500mg.
        """,
    "texte3" : """Admission d’un homme de 72ans le 02/02/22.
        Douleurs thoraciques. ECG normal. Troponine: 0.04ng/mL.
        Ttt: surveillance + aspirine 75mg/j.
        """,
    "texte4" : """Pat. 25ans, suivi pour asthme. Crise aiguë le 04/03/2025.
        Oxygène à 3L/min. Corticoïdes 100mg IV administrés.
        """,
    "texte5" : """Pat. 50ans diabétique. Consultation du 10/10/2020.
        HbA1c: 9.2%. Ttt modifié: metformine 1000mg/j.
        Régime conseillé. Poids: 95kg.
        """
}


for sen in data.values():
    tokens = supprimer_stopwords(tokenisation(sen))
    normalized = normalisation(tokens)
    print(normalized)

['patient', '60', 'an', 'hospitalis', 'pneumopath', 'crp', '240', 'mg', '/', 'l', 'trait', 'initial', 'amoxicillin', '1', 'g', '/', 'j', 'fievr', 'persist', '38.', 'c', 'scann', 'réalis']
['patient', '33', 'an', 'chut', 'scann', 'cérébral', 'normal', 'températur', '37.8ºc', 'trait', 'paracétamol', '500', 'mg']
['admiss', 'dun', 'homm', '72', 'an', 'douleur', 'thorac', 'ecg', 'normal', 'troponin', 'o.o4', 'ng', '/', 'ml', 'trait', 'surveil', 'aspirin', '75', 'mg', '/']
['patient', '25', 'an', 'suiv', 'asthme', 'cris', 'aigu', 'oxygen', '3', 'l', '/', 'min', 'corticoïd', '100', 'mg', 'intravein', 'administr']
['patient', '50', 'an', 'diabet', 'consult', 'hbal', '9.2', 'trait', 'modifi', 'metformin', '1000', 'mg', '/', 'régim', 'conseil', 'poid', '95', 'kg']
